# INITIAL SETUP

In [ ]:
from math import pi
import numpy as np

# Oh yeah, it's all coming together...
from CargoBikeModel import *
from ModelAnalysisFunctions import *

In [ ]:
version = '45'                                  # Version control for the results
bike_floor_height = np.arange(0.12, 0.28, 0.01) # Bike floor height range -> 0.15m to 0.27m, with 0.01m step
cargo_space = np.arange(0.4, 1.55, 0.05)        # Cargo space range -> 0.5m to 1.5m, with 0.05m step
steering_angle = np.arange(15, 30, 1)*pi/180    # Steering angle range -> 18° to 30°, with 1° step

# DEFINING PARAMETERS DICTIONARIES

### FRAME PARAMETERS

In [ ]:
L = [                         # Frame's bars' lengths
     0.45,                    # Bars 1-2
     0.48,                    # Bars 1-3
     0.43,                    # Bar 2-3
     0.48,                    # Bar 2-4
     0.00, # ////////////////// Bar 3-5 -> Tied to N3 and N5 (calculated by cbm function)
     0.50,                    # Bar 4-5
     cargo_space[0],          # Bar 4-6
     0.00, # ////////////////// Bar 6-7 -> Tied to lam and w (calculated by cbm function)
     0.35,                    # Bars 7-8
     0.03  # ////////////////// Bars 8-9 -> Tied to trail    (used in get_trail)
    ]
W = [                         # Frame's bars' wall thicknesses
     0.003,                   # Bars 1-2
     0.003,                   # Bars 1-3
     0.003,                   # Bar 2-3
     0.003,                   # Bar 2-4
     0.003,                   # Bar 3-5
     0.003,                   # Bar 4-5
     0.003,                   # Bar 4-6
     0.003,                   # Bar 6-7
     0.003,                   # Bars 7-8
     0.003                    # Bars 8-9
    ]
R = [                         # Frame's bars' radiuses
     merge_bars(0.015, W[0]), # Bars 1-2 -> Merged for the virtual model
     merge_bars(0.014, W[1]), # Bars 1-3 -> Merged for the virtual model
     0.019,                   # Bar 2-3
     0.018,                   # Bar 2-4
     0.018,                   # Bar 3-5
     0.020,                   # Bar 4-5
     0.018,                   # Bar 4-6
     0.018,                   # Bar 6-7
     merge_bars(0.012, W[8]), # Bars 7-8 -> Merged for the virtual model
     merge_bars(0.012, W[9])  # Bars 8-9 -> Merged for the virtual model
    ]

r_RW = 0.28    # Rear wheel radius
r_FW = 0.20    # Front wheel radius
m_RW = 2.6 + 5 # Rear wheel mass + Electric motor mass
m_FW = 2.0     # Front wheel mass

h_RF = bike_floor_height[0]   # Rear frame height / Bike floor height

lam = steering_angle[0]                      # Lambda / Steering angle
c = get_trail(steering_angle[0], r_FW, L[9]) # Trail

d_bw = 0.2*r_FW                                                    # Front wheel distance to the bar 6-7
rear_length = get_rear_length(h_RF, r_RW, L[0], L[3])              # Horizontal distance between N1 and N4
front_length = get_front_length(h_RF, lam, d_bw, r_FW, L[8], L[9]) # Horizontal distance between N4 and N9
w = rear_length + cargo_space[0] + front_length                    # Wheelbase

# Frame + Wheels dictionary
frame_params = { 
    'rear_wheel_radius': r_RW,
    'front_wheel_radius': r_FW,
    'rear_frame_height': h_RF,
    'wheelbase': w,
    'steer_axis_tilt': lam,
    'trail': c,
    'bars_lengths': L,
    'rear_wheel_mass': m_RW,
    'front_wheel_mass': m_FW,
    'material_density': 2.7e3, # Frame's material density (Al6061)
    'bars_radiuses': R, 
    'bars_thicknesses': W,
    'd_bw': d_bw
}

### RIDER PARAMETERS

In [ ]:
height = 1.71
weight = 72.7

# For the body proportions, we're considering that a
# standing human is as tall as 7.5 stacked human heads!
head = height/7.5

# Rider dictionary
rider_params = {
    'rider_name': 'Alexander',
    'saddle_height': 0.15,
    'handlebar_height': 0.15,
    'lean_angle': 75*pi/180,   # How much the rider is leaning foward

    'limbs_lengths': [
        2*head,         # Shanks     (1-2)
        2*head,         # Thighs     (2-3)
        2.75*head,      # Torso      (3-4)
        1.5*head,       # Upper arms (4-5)
        1.5*head,       # Forearms   (5-6)
        1*head          # Head       (4-7)
    ],

    # Mass distribuded using Yeadon's library documentation!
    # https://yeadon.readthedocs.io/en/latest/
    'limbs_masses': [
        0.061*2*weight, # Shanks     (1-2)
        0.100*2*weight, # Thighs     (2-3)
        0.510*weight,   # Torso      (3-4)
        0.028*2*weight, # Upper arms (4-5)
        0.022*2*weight, # Forearms   (5-6)
        0.068*weight    # Head       (4-7)
    ] 
}

### CARGO PARAMETERS

In [ ]:
cargo_density = 167 # The cargo's density [kg/m^3]
cargo_height = 0.5  # The cargo's vertical length in the z-axis
cargo_width = 0.5   # The cargo's horizontal length in the y-axis

# Cargo dictionary
cargo_params = {
    'cargo_height': cargo_height,
    
    # Function for the cargo's mass, in case it varies with the cargo space length
    'mass_function': lambda cargo_length: cargo_density*cargo_height*cargo_width*cargo_length
}

# The cargo's mass is calculated with mass_function(cargo_length),
# where cargo_length/cargo_space is the cargo's horizontal lenght in the x-axis
cargo_params['cargo_mass'] = cargo_params['mass_function'](cargo_space[0])

# BICYCLE MODELS ITERATIONS

### SETUP 1

In [ ]:
# System parameters for the setup 1
system_params = {'frame': frame_params,
                 'rider': rider_params}

results_setup1 = simulate_bikes(system_params, bike_floor_height, cargo_space, steering_angle)
save_results(results_setup1, 'ar_' + version + 'setup1')

### SETUP 2

In [ ]:
# System parameters for the setup 2
cargo_params['mass_function'] = lambda cargo_length: cargo_density*cargo_height*cargo_width*cargo_length
system_params = {'frame': frame_params,
                 'rider': rider_params,
                 'cargo': cargo_params}

results_setup2 = simulate_bikes(system_params, bike_floor_height, cargo_space, steering_angle)
save_results(results_setup2, 'ar_' + version + 'setup2')

### SETUP 3

In [ ]:
# System parameters for the setup 3
cargo_params['mass_function'] = lambda cargo_length: 40
system_params = {'frame': frame_params,
                 'rider': rider_params,
                 'cargo': cargo_params}

results_setup3 = simulate_bikes(system_params, bike_floor_height, cargo_space, steering_angle)
save_results(results_setup3, 'ar_' + version + 'setup3')

# BIKE SANDBOX

To test and visualize different parameters for the bike

In [ ]:
# Parameters to change:
L[9] = 0.03
h_RF = 0.15
L[6] = 0.70
lam = 16*pi/180
cargo_params['cargo_height'] = 0.5
cargo_params['mass_function'] = lambda cargo_length: cargo_density*cargo_height*cargo_width*cargo_length

# Updates dictionaries with new parameters
rear_length = get_rear_length(h_RF, r_RW, L[0], L[3])
front_length = get_front_length(h_RF, lam, 0.2*r_FW, r_FW, L[8], L[9])
frame_params['rear_frame_height'] = h_RF
frame_params['bars_lengths'][6] = L[6]
frame_params['wheelbase'] = rear_length + L[6] + front_length
frame_params['steer_axis_tilt'] = lam
frame_params['trail'] = get_trail(lam, r_FW, L[9])
cargo_params['cargo_mass'] = cargo_params['mass_function'](L[6])

# Creates bike object
cargo_bike = CargoBike('Cargo', {'frame': frame_params, # })
                                 'rider': rider_params, # })
                                 'cargo': cargo_params})

# Shows bike geometry
figure = cargo_bike.show()

In [ ]:
# Creates a figure for the graph
figure = plt.figure(figsize=(16, 8)) # Adjustable window size
plt.xlim(0, 15)                      # Adjustable horizontal view
plt.ylim(-5, 5)                      # Adjustable vertical view

# Plots the eigenvalues vs speed graph of the bike
# Method inherited from the class Bicycles, from the package BicycleParameters!
# https://pythonhosted.org/BicycleParameters/index.html#
cargo_bike.plot_eigenvalues_vs_speed(np.linspace(0., 48., num=100), fig=figure)

plt.show()

# Prints the speed values within the stable speed range
print('We found these speeds withing the stable speed range:')
print(get_stable_speeds(cargo_bike))

In [ ]:
# Saves a dictionary to a .json file
save_parameters(frame_params, 'frame_dict')